In [4]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [7]:

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100.0%


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [8]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [9]:
net = Net().to(device)

In [10]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 1.062
[1,   200] loss: 0.644
[1,   300] loss: 0.592
[1,   400] loss: 0.540
[1,   500] loss: 0.498
[1,   600] loss: 0.465
[1,   700] loss: 0.472
[1,   800] loss: 0.449
[1,   900] loss: 0.434
[2,   100] loss: 0.413
[2,   200] loss: 0.396
[2,   300] loss: 0.380
[2,   400] loss: 0.380
[2,   500] loss: 0.369
[2,   600] loss: 0.364
[2,   700] loss: 0.350
[2,   800] loss: 0.352
[2,   900] loss: 0.359
[3,   100] loss: 0.341
[3,   200] loss: 0.328
[3,   300] loss: 0.319
[3,   400] loss: 0.314
[3,   500] loss: 0.329
[3,   600] loss: 0.331
[3,   700] loss: 0.309
[3,   800] loss: 0.314
[3,   900] loss: 0.320
[4,   100] loss: 0.290
[4,   200] loss: 0.290
[4,   300] loss: 0.284
[4,   400] loss: 0.303
[4,   500] loss: 0.295
[4,   600] loss: 0.289
[4,   700] loss: 0.294
[4,   800] loss: 0.278
[4,   900] loss: 0.299
[5,   100] loss: 0.258
[5,   200] loss: 0.272
[5,   300] loss: 0.268
[5,   400] loss: 0.277
[5,   500] loss: 0.259
[5,   600] loss: 0.264
[5,   700] loss: 0.262
[5,   800] 

In [12]:

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [13]:
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 90 %
